# Visualización de datos.
#### En este documento se visualizarán los datos de tráfico, tratando de capturar información importante.

Primero se imprtan las librerías necesarias para el correcto funcionamiento del cuaderno de Jupyter.

In [ ]:
# Librerías estándar
from datetime import datetime  # Manejo de fechas y tiempos

# Manejo de datos
import pandas as pd  # Manipulación de datos tabulares

# Visualización
import matplotlib.pyplot as plt  # Visualización básica
import seaborn as sns  # Visualización estadística con estilo

# Análisis de series temporales
from statsmodels.tsa.seasonal import seasonal_decompose  # Descomposición de series temporales

Cargamos los datos

In [ ]:
df = pd.read_csv('../data/2018-2025 traffic.csv', parse_dates=['fecha'])

Pasamos a numéricas aquellas columnas que contengan números.

In [ ]:
cols_num = ['intensidad', 'ocupacion', 'carga', 'vmed', 'periodo_integracion']

for col in cols_num:
    df[col] = pd.to_numeric(df[col], errors='coerce')

Eliminams valores nulos.

In [ ]:
df = df.dropna()

Vemos los resultados.

In [ ]:
df.head()

Ponemos la fecha como índice en una copia del DataFrame.

In [ ]:
# Convertir la columna 'fecha' a tipo datetime
df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce')
df = df.dropna(subset=['fecha'])
df = df.sort_values('fecha')

# Crear una copia del DataFrame y establecer 'fecha' como índice
df_copia = df.copy()
df_copia.set_index('fecha', inplace=True)

In [ ]:
df_copia.head()

## 1. Descomposición de la serie temporal (Tendencia, Estacionalidad, Residuos)
Esto ayuda a entender la estructura de la serie temporal para cada punto.

Primero nos aseguramos que el índice es tipo Datetime.

In [ ]:
print(df_copia.index)

Eliminamos los posibles nulos que haya y los agrupamos.

In [ ]:
df_copia = df_copia[~df_copia.index.isna()]
df_agrupado = df_copia.groupby(df_copia.index).mean(numeric_only=True)

Descomoponemos la serie mensualmente.

In [ ]:
# Agrupar mensualmente
df_monthly = df_agrupado.resample('ME').mean(numeric_only=True)
df_monthly['intensidad'].dropna().plot(title='Intensidad Mensual')

# Descomposición
decomp = seasonal_decompose(df_monthly['intensidad'].dropna(), model='additive', period=12)

# Plot
decomp.plot()
plt.suptitle(f"Descomposición Serie Temporal Mensual", y=1.05)
plt.tight_layout()
plt.show()

## 2. Comportamiento diario promedio (Media de intensidad por hora)
Para visualizar patrones diarios típicos de tráfico:

In [ ]:
# Extraemos hora
df['hora'] = df['fecha'].dt.hour

# Promedio por hora para todos los puntos
hora_media = df.groupby('hora')['intensidad'].mean().reset_index()

# Plot
plt.figure(figsize=(10, 5))
sns.lineplot(data=hora_media, x='hora', y='intensidad')
plt.title('Promedio de Intensidad por Hora del Día (Todos los Puntos)')
plt.xlabel('Hora del Día')
plt.ylabel('Intensidad Promedio')
plt.grid(True)
plt.show()

## 3. Boxplot por día de la semana y hora
Permite ver la variabilidad del tráfico en distintas franjas horarias durante la semana.

In [ ]:
df['dia_semana'] = df['fecha'].dt.day_name()

plt.figure(figsize=(14, 6))
sns.boxplot(x='hora', y='intensidad', hue='dia_semana', data=df, showfliers=False)
plt.title('Distribución de Intensidad por Hora y Día de la Semana')
plt.xlabel('Hora del Día')
plt.ylabel('Intensidad')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

## 4. Correlación entre variables
Esto puede ayudar a decidir qué variables incluir en el modelo.

In [ ]:
corr = df[['intensidad', 'ocupacion', 'carga', 'vmed']].corr()

sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title('Correlación entre Variables de Tráfico')
plt.show()